In [1]:
import random
import numpy as np
import tensorflow as tf

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]

In [15]:
characters = sorted(set(text))
# print(characters)


char_to_index = dict((c, i) for i, c in enumerate(characters))
# print(char_to_index)


index_to_char = dict((i, c) for i, c in enumerate(characters))
# print(index_to_char)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}


In [5]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

# sentences will have sentences and
# next_char will try to predict next word in sentences 

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])
    
    # print(next_char)
    
    #simply this loop is breaking all the words in chars
    #Eg :=.   
    #    next_char = ['Hello', 'ello,', 'llo, ', 'lo, h', 'o,
    #     ho', ', how', ' how ', 'how a', 'ow ar', '
    #     w are', ' are ', 'are y', 're yo', 'e you', ' you', 'you ', 
    #    'ou to', 'u tod', ' toda', 'today']



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [7]:
#  code is creating a data representation for 
# training a neural network. It encodes sequences of 
# characters (x) and the next character after each sequence (y) 
# in a binary format, where 1 indicates the presence of a 
# character at a specific position.

x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=bool)


for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1
    

    


# making neural network

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [9]:
model = Sequential()
#sequential?? a simple stack layer where 
# each layer has one input tensor and one output tensor



model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))
#adding a lstm layer with 128neurons to model

model.add(Dense(len(characters)))
# This specifies a dense layer with a number of units
# equal to the length of the characters
# character=['\n', ' ', '!', '$', '&', "'", ',', '-', '.',
#            '3', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 
#            'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
# The number of units in the dense layer often 
# corresponds to the number of classes or unique
# elements in the output.


model.add(Activation('softmax'))
# it takes a set of numbers and converts 
# them into a probability distribution,
# meaning it makes them look like probabilities.

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 [==============================] - 223s 292ms/step - loss: 2.6923
Epoch 2/4
651/651 [==============================] - 167s 256ms/step - loss: 2.2990
Epoch 3/4
651/651 [==============================] - 141s 217ms/step - loss: 2.1811
Epoch 4/4
651/651 [==============================] - 102s 157ms/step - loss: 2.0999


In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# generating

In [12]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [13]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))

ermione:
take the boy to you: he so trough mand of the the the to the mand and and and the fored in the with the pronter and the sore the the the the the the doren and the the the the the ward the wart in that the pranter of the the the the the for the the the with the mand and and and and the wardes for the dound hath the there the ware 
ts; therefore, be bold.

northumberland:
what the the thes i lave the grownis do the treen.

nore:
in the sarke thou the wart and with in here porese my word in the reant.

kender:
and and are the will so merest the shath and some that bather wart but the for the sout and of the freth here wall is nous of the wing weat the ward mand and t
ou, on my life, and holds you dear
as havent in to his the for the wall the mound in the sore in beis of is for made of agling of hos ow the dorent.

king ard in this mand werrdend hard some il no saen his forle the keand for urence the the beand
nourthe book mand as mane,
here the thar are the care the cavess frath,